In [1]:
import os
import numpy as np

os.chdir('/Users/osono/Library/CloudStorage/OneDrive-NorthCarolinaA&TStateUniversity/COMP 850 - 1A Big Data Analytics/Project/continuous authentication/datasets/Behacom')
current_directory = os.getcwd()
print(f'changed working dir to: {current_directory}')

file_paths = np.array([])
for root, dirs, files in os.walk(current_directory):
    for file in files:
        if file.endswith('.csv'):
            file_paths = np.append(file_paths, os.path.join(root, file))
            
while file_paths.size > 0:
   choices =  np.random.choice(file_paths, size=2, replace=False)
   file_paths = file_paths[np.isin(file_paths, choices, invert=True)]


changed working dir to: /Users/osono/Library/CloudStorage/OneDrive-NorthCarolinaA&TStateUniversity/COMP 850 - 1A Big Data Analytics/Project/continuous authentication/datasets/Behacom


In [2]:
import pandas as pd

#import data

normal = pd.read_csv('User0/User0_BEHACOM.csv', encoding='latin-1') 
anomalies = pd.read_csv('User2/User2_BEHACOM.csv', encoding='latin-1') 
timestamp = 'timestamp'
unix_timestamp = 'unix_timestamp'
normal['USER'] = 0
anomalies['USER'] = 1
normal[unix_timestamp], normal[timestamp] = normal[timestamp], pd.to_datetime(normal[timestamp], unit='ms')
anomalies[unix_timestamp], anomalies[timestamp] = anomalies[timestamp], pd.to_datetime(anomalies[timestamp], unit='ms')
print(normal.shape, anomalies.shape)

(6059, 12052) (179, 12052)


In [3]:
from  datetime import timedelta, datetime



def random_time_range(anomalies: pd.DataFrame, iteration: int) -> tuple[datetime, datetime]:
    #determine start and end time. subtrat one second from start time to ensure the real start time is always included
    start_time, end_time = anomalies[timestamp].min() - timedelta(seconds=1) , anomalies[timestamp].max()
    print(f'anomalies start_time: {start_time} - end_time: {end_time}')
    
    #pick a random time delta between a range, weighted by the number of iterations
    delta = timedelta(minutes= np.random.randint(5,30) + 2 ** iteration)
    latest_start  = np.maximum(end_time - delta, start_time)
        
    random_start = start_time + timedelta(seconds=np.random.randint(0, int((latest_start - start_time).total_seconds() + 1))) 
    random_end = random_start + delta
    print(f'random time rand selected {random_start} - {random_end}')
    return random_start, random_end

def inject_anomalies(normal:pd.DataFrame, anomalies:pd.DataFrame, start_time: datetime, end_time:datetime) -> tuple[pd.DataFrame, pd.Index]:
    anomalies_condition = (start_time <= anomalies[timestamp]) & (anomalies[timestamp] <= end_time)
    normal_conditoin = (start_time <= normal[timestamp]) & (normal[timestamp] <= end_time)
    #find anomalies that fall within the random time range
    subset_anomalies = anomalies[anomalies_condition]
    index = subset_anomalies.index
    print(f'{subset_anomalies.shape[0]} anomalies selected')

    #if no anomalies found do nothing
    if subset_anomalies.empty:
        return normal, index
    
    #find normals that fall within the same random time range
    subset_normal = normal[normal_conditoin]
    
    #if no normals found append anomalies to normals list
    if subset_normal.empty:
        print('no normal data within range, appending anomalies')
        return pd.concat([normal,subset_anomalies], ignore_index=True), index
    
    #if normals found remove them and append anomalies to list to prevent unrealistic data where 2 users are interating with 1 machine
    print(f'{subset_normal.shape[0]} normals selected to be overwritten')
    return pd.concat([normal.drop(normal[normal_conditoin].index), subset_anomalies], ignore_index=True), index


In [4]:
anomaly_percentage_target = .3
max_size = normal.shape[0] + anomalies.shape[0]

anomaly_percentage = 0
iteration = 0
merged, a = normal.copy(), anomalies.copy()
#merge two datasets
while anomaly_percentage < anomaly_percentage_target and merged.shape[0] < max_size and not a.empty:
    iteration+=1
    print(f'current iteration: {iteration} - anomaly_size: {a.shape} - anomaly_percentage: {anomaly_percentage} - current_size: {merged.shape[0]} -  max_size: {max_size}')
    print(f'starting shape: {merged.shape}')
    merged, index = inject_anomalies(merged, a, *random_time_range(a, iteration))
    a.drop(index, inplace=True)
    print(f'ending shape: {merged.shape}')
    anomaly_percentage = merged[merged['USER'] == 1].shape[0] / merged.shape[0]

print(f'total iterations: {iteration} - final anomaly_percentage: {anomaly_percentage}')
merged.sort_values(timestamp)


current iteration: 1 - anomaly_size: (179, 12052) - anomaly_percentage: 0 - current_size: 6059 -  max_size: 6238
starting shape: (6059, 12052)
anomalies start_time: 2019-12-02 20:39:21.282000 - end_time: 2019-12-02 23:55:05.585000
random time rand selected 2019-12-02 22:18:54.282000 - 2019-12-02 22:36:54.282000
16 anomalies selected
no normal data within range, appending anomalies
ending shape: (6075, 12052)
current iteration: 2 - anomaly_size: (163, 12052) - anomaly_percentage: 0.0026337448559670784 - current_size: 6075 -  max_size: 6238
starting shape: (6075, 12052)
anomalies start_time: 2019-12-02 20:39:21.282000 - end_time: 2019-12-02 23:55:05.585000
random time rand selected 2019-12-02 22:55:53.282000 - 2019-12-02 23:25:53.282000
28 anomalies selected
no normal data within range, appending anomalies
ending shape: (6103, 12052)
current iteration: 3 - anomaly_size: (135, 12052) - anomaly_percentage: 0.00720956906439456 - current_size: 6103 -  max_size: 6238
starting shape: (6103, 12

,timestamp,keystroke_counter,erase_keys_counter,erase_keys_percentage,press_press_average_interval,press_press_stddev_interval,press_release_average_interval,press_release_stddev_interval,word_counter,word_average_length,...,system_average_cpu,system_stddev_cpu,current_app_average_mem,current_app_stddev_mem,system_average_mem,system_stddev_mem,received_bytes,sent_bytes,USER,unix_timestamp
0,2019-11-20 13:44:15.128,14,2,0.14,200.69,165.47,107.00,36.45,2,5.00,...,13.03,14.42,11.07,0.00,1.15,2.57,1618568.0,274869.0,0,1.574257e+12
1,2019-11-20 13:45:18.073,37,3,0.08,825.69,1842.59,132.68,96.69,2,10.50,...,17.50,6.45,1.44,0.08,12.71,0.97,14835273.0,449913.0,0,1.574258e+12
2,2019-11-20 13:46:18.881,24,0,0.00,2399.87,5181.55,122.22,28.14,3,5.00,...,19.73,4.89,11.66,0.56,16.50,6.15,5097547.0,272460.0,0,1.574258e+12
3,2019-11-20 13:47:19.398,28,0,0.00,2087.78,4272.74,308.11,272.18,1,3.00,...,13.60,3.18,12.55,1.31,25.07,2.55,3926179.0,293861.0,0,1.574258e+12
4,2019-11-20 13:48:20.304,71,6,0.08,421.29,1260.38,153.82,138.59,5,11.20,...,16.09,8.56,12.89,1.32,25.72,2.54,2106149.0,241120.0,0,1.574258e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6001,2020-01-11 18:03:03.150,8,0,0.00,580.43,851.32,87.38,28.61,1,4.00,...,5.21,2.45,17.50,0.28,18.98,0.28,1447400.0,234625.0,0,1.578766e+12
6002,2020-01-11 18:04:05.097,32,3,0.09,1441.19,6028.06,146.71,140.72,6,3.50,...,6.42,3.39,16.47,0.32,17.96,0.32,29698808.0,526419.0,0,1.578766e+12
6003,2020-01-11 18:05:07.518,12,0,0.00,214.82,191.39,116.75,30.17,2,4.50,...,7.65,1.78,15.55,0.93,17.04,0.93,31654926.0,1139532.0,0,1.578766e+12
6004,2020-01-11 18:06:09.202,65,0,0.00,208.12,682.38,117.16,26.79,13,4.08,...,5.46,2.45,16.59,1.04,18.08,1.04,4487052.0,313436.0,0,1.578766e+12


In [5]:
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler



y = merged.iloc[:,merged.columns.isin(['USER'])].to_numpy()
print(y.shape)
#reduce dimensions of the data to focus only on keyboard and mouse telemetry
X = merged.iloc[:, ~merged.columns.isin([timestamp, 'USER', 'active_apps_average',
'current_app',
'penultimate_app',
'changes_between_apps',
'current_app_foreground_time',
'current_app_average_processes',
'current_app_stddev_processes',
'current_app_average_cpu',
'current_app_stddev_cpu',
'system_average_cpu',
'system_stddev_cpu',
'current_app_average_mem',
'current_app_stddev_mem',
'system_average_mem',
'system_stddev_mem',
'received_bytes',
'sent_bytes'])].to_numpy()
print(X.shape)


(6061, 1)
(6061, 12033)


In [6]:
#normalizing data 
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [7]:
#split data
from sklearn.model_selection import train_test_split 
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled,y, train_size=0.7, test_size=0.3, shuffle=False, random_state=42) 

# Split the temporary data into 50% validation and 50% test (50% of 30% is 15% each)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)



In [9]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
look_back = 3
batch_size = 45
n_features = X_train.shape[1]
train_generator = TimeseriesGenerator(X_train, y_train, length=look_back, batch_size=batch_size)
test_generator = TimeseriesGenerator(X_test, y_test, length=look_back, batch_size=batch_size)
val_generator = TimeseriesGenerator(X_val, y_val, length=look_back, batch_size=batch_size)


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(look_back, n_features)))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

/opt/homebrew/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 50)             │     2,416,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,416,851 (9.22 MB)

 Trainable params: 2,416,851 (9.22 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
history = model.fit(train_generator, epochs=2, verbose=1)
type(history)



Epoch 1/2
 4/95 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.9528 - loss: 0.5213   

/opt/homebrew/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


95/95 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.8710 - loss: 2.0599
Epoch 2/2
95/95 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.9828 - loss: 0.2322


keras.src.callbacks.history.History

In [12]:

test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print(f'Test Accuracy: {test_acc *100:.2f}%')


21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9249 - loss: 9.7678 
Test Accuracy: 92.50%


In [14]:
from sklearn.metrics import accuracy_score,mean_absolute_error, root_mean_squared_error, confusion_matrix


# Make a prediction (example with last batch of validation data)
predictions = model.predict(val_generator)
predictions = (predictions >= 0.5).astype(int)
actual = val_generator.targets[look_back:]

# Calculate accuracy, MAE and RMSE
accuracy = accuracy_score(actual, predictions)
mae = mean_absolute_error(actual, predictions)
rmse = root_mean_squared_error(actual, predictions)
true_negatives, false_positives, false_negatives, true_positives = confusion_matrix(actual, predictions).ravel()
total = actual.shape[0]
false_positives_rate = false_positives / total
false_negatives_rate = false_negatives / total

print(f'Accuracy: {accuracy:.2f}')
print(f'Mean Absolute Error: {mae:.2f}')
print(f'Root Mean Squared Error: {rmse:.2f}')
print(f'False Positives: {false_positives_rate:.2f}')
print(f'Fales Negatives: {false_negatives_rate: .2f}')
print(true_negatives, true_positives, false_negatives, false_positives)

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Accuracy: 0.92
Mean Absolute Error: 0.08
Root Mean Squared Error: 0.28
False Positives: 0.05
Fales Negatives:  0.03
835 0 26 45
